In [19]:
import time
import pandas as pd
import numpy as np
from datetime import datetime
import re
import locale
import pyautogui

from utils import *

# Importing Selenium library and relevant classes
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains

Define: ```Elemento a buscar```
 ```Lista a incluir``` 
 ```Lista a excluir```

In [ ]:
elemento_a_buscar = 'iphone 16'
#funcion para seleccionar los articulos que quiero:
# si una la tiene o no la tiene se incluye o excluye.
# por ejemplo si buscadas = 15 y 16
#                excluyentes = funda cristal
#                'funda iphone 15' se excluye
#                'iphone'      no se incluye porque no tiene el 15 ni el 16
cadenas_buscadas = (
                    "16",
                    "16."
                    )  # la coma es para que python lo vea como una tupla
cadenas_excluyentes = (
                    "funda",
                    "carcasa",
                    "cristal",
                    "4"
)
#selecciona SOLO UNO. No modificar los nombres
estado  = ( 
            'nuevo'
            #'como nuevo'
            #'en buen estado'
            )

#n_scrolls_total = 40 # no se usa, numero de veces que haremos scroll abajo. Cada 10 carga datos
n_scrolls_cada_vez = 25 # cuantos scrolls hace cada vez
if type(cadenas_buscadas) is not tuple:
    print("cadenas_buscadas no es tupla")
if type(estado) is not str:
    print("estado no es str")  

In [21]:
#Abre la venana
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

driver.get('https://es.wallapop.com');
time.sleep(1)
driver.maximize_window()  
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(1)

Let's accept the website cookies that are shown once the page loads.

Sometimes the page loads without the requirement to accept the cookies, and therefore the code to accept cookies has to be placed in a try-except statement

In [22]:
WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()


In [23]:
# Busca el articulo
search_box = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.ID, "searchbox-form-input"))
)
search_box.send_keys(elemento_a_buscar)
search_box.submit()

In [24]:
time.sleep(6)
# clico 3 veces fuera del cuadrado para saltar el tutorial
x_coord = 300  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(2)
x_coord = 300  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(2)
x_coord = 300  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(4)

In [25]:
# no se que hace. creo que cargar 40 articulos más
# button = WebDriverWait(driver, 6).until(
#     EC.element_to_be_clickable((By.XPATH, "//walla-button[contains(@class, 'w-100')]"))
# )
# # Haz clic en el botón
# button.click()
# time.sleep(2)

In [26]:
# Abro y busco el campo de distancia
madrid_div = driver.find_element(By.CSS_SELECTOR, '.Bubble__content')
madrid_div.click()

In [27]:
#seleciono el desplegable de Madrid
try: 
     element = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'España, Madrid')]"))
     )
     element.click()
except:
     print("reintento seleccionar el desplegable de Madrid")
     time.sleep(2)
     element = WebDriverWait(driver, 10).until(
     EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'España, Madrid')]"))
     )
     element.click()

In [28]:
#Selecciono los km del 

time.sleep(3)
# 30km. Si ejecutas este chunk, colocará el mouse en las coordenadas marcadas
x_coord = 250 
y_coord = 780 
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(3)


In [29]:
button = WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn--width btn-filter btn-primary']"))
)
button.click()
time.sleep(2)


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x007BEBD3+24307]
	(No symbol) [0x00748D74]
	(No symbol) [0x0062C323]
	(No symbol) [0x0066DC86]
	(No symbol) [0x0066DECB]
	(No symbol) [0x006AB9F2]
	(No symbol) [0x0068FED4]
	(No symbol) [0x006A9579]
	(No symbol) [0x0068FC26]
	(No symbol) [0x0066219C]
	(No symbol) [0x0066311D]
	GetHandleVerifier [0x00A68D93+2818227]
	GetHandleVerifier [0x00AC542E+3196750]
	GetHandleVerifier [0x00ABD9D2+3165426]
	GetHandleVerifier [0x0085DA70+675216]
	(No symbol) [0x00751B3D]
	(No symbol) [0x0074EA18]
	(No symbol) [0x0074EBB5]
	(No symbol) [0x00741640]
	BaseThreadInitThunk [0x76127BA9+25]
	RtlInitializeExceptionChain [0x7721C0CB+107]
	RtlClearBits [0x7721C04F+191]


In [ ]:
# Filtro por estado
driver.implicitly_wait(10)
elemento = driver.find_element(By.XPATH, "//span[contains(text(), 'Estado del producto')]")
elemento.click()

In [ ]:
time.sleep(2)
x_coord = 585 
offset = 42
if (estado == "nuevo"):
    y_coord = 380 + offset
if (estado == "como nuevo"):
    y_coord = 460 + offset
if (estado == "buen estado"):
    y_coord = 550 + offset
pyautogui.click(x=x_coord, y=y_coord)


In [ ]:
# Creación de la tabla con las columnas que buscamos
try:
    del df
except NameError:
    pass
columnas = ['id', 'nombre', 'precio', 'estado', 'reservado', 'url']
df = pd.DataFrame(columns=columnas)

In [ ]:
time.sleep(2)
x_coord = 1600
y_coord = 780 
pyautogui.click(x=x_coord, y=y_coord)

# Scrolling

In [ ]:
# Solo vemos 40. Clicamos para ver más y hacemos scroll down n veces
boton = driver.find_element(By.ID, 'btn-load-more')
boton.click()
n = 0
while n < n_scrolls_cada_vez:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    n = n + 1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="btn-load-more"]"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x007BEBD3+24307]
	(No symbol) [0x00748D74]
	(No symbol) [0x0062C323]
	(No symbol) [0x0066DC86]
	(No symbol) [0x0066DECB]
	(No symbol) [0x006AB9F2]
	(No symbol) [0x0068FED4]
	(No symbol) [0x006A9579]
	(No symbol) [0x0068FC26]
	(No symbol) [0x0066219C]
	(No symbol) [0x0066311D]
	GetHandleVerifier [0x00A68D93+2818227]
	GetHandleVerifier [0x00AC542E+3196750]
	GetHandleVerifier [0x00ABD9D2+3165426]
	GetHandleVerifier [0x0085DA70+675216]
	(No symbol) [0x00751B3D]
	(No symbol) [0x0074EA18]
	(No symbol) [0x0074EBB5]
	(No symbol) [0x00741640]
	BaseThreadInitThunk [0x76127BA9+25]
	RtlInitializeExceptionChain [0x7721C0CB+107]
	RtlClearBits [0x7721C

In [ ]:
a

In [ ]:
a
#Reservado
def item_reservado(id, df):
    try:
        # Buscar el item en el DataFrame por el id
        item = df[df['id'] == id]
        
        if not item.empty:
            # Construir el selector CSS basado en el id
            selector = f"body > tsl-root > tsl-public > div > div > tsl-search-root > div > tsl-search-unified > div > tsl-search-layout-sidebar > div > div.col.SearchLayoutSidebar__main > div:nth-child({id}) > div:nth-child(1) > section > div > tsl-public-item-card-list > div > a:nth-child(205) > tsl-public-item-card > div > div.position-absolute.d-flex.flex-column.justify-content-center.align-items-center"
            
            # Encontrar el elemento en la página web
            elemento = driver.find_element(By.CSS_SELECTOR, selector)
            
            # Verificar si el elemento tiene la clase "seleccionado" (ajusta esto según el estado)
            # Este es un ejemplo que depende de un cambio de clase. Ajusta según lo que se use en la página web
            if 'seleccionado' in elemento.get_attribute('class'):
                return 1
            else:
                return 0
        else:
            print(f"El id {id} no se encontró en el DataFrame.")
            return 0


In [ ]:
# Extraigo los datos y los añado en la tabla. 
elementos = driver.find_elements(By.CSS_SELECTOR, "a.ItemCardList__item")

for index, elem in enumerate(elementos):
    nombre = elem.get_attribute('title')
    nombre = nombre.lower()
    if existe_cadena(nombre, cadenas_buscadas, cadenas_excluyentes):
        url_articulo = elem.get_attribute('href')
        id_articulo = url_articulo.split('-')[-1]
        precio = elem.find_element(By.CSS_SELECTOR, ".ItemCard__price").text.strip()
        precio = precio.split(' ')[0]
        
        if ya_existe_articulo(id_articulo, df):
            print(nombre, " ya existe con id: ", id_articulo)
        else:
            df.loc[index] = [id_articulo, nombre, precio, estado, url_articulo]
    else:
        print(nombre, "### excluido ###")
df = df.reset_index(drop=True)
print(df)

funda para iphone 16 excluido
funda para iphone 16 excluido
funda para iphone 16 pro excluido
iphone 16pro max 256gb  excluido
funda y protector iphone 16 pro max excluido
funda iphone 16promax  excluido
iphone16pro excluido
funda magnetica iphone 16 pro excluido
funda magnetica iphone 16 promax excluido
iphone 13 128gb  excluido
funda iphone 16 pro excluido
iphone 14 plus excluido
funda iphone 16 pro excluido
funda cuero  magnetica con magsafe iphone 16 pro excluido
funda de silicona magsafe original para iphone 16  excluido
iphone16plus excluido
iphone excluido
iphone 5s excluido
funda magsafe iphone 16 pro excluido
funda personalizable iphone 11-16 excluido
funda magsafe iphone 16 pro excluido
funda silicona iphone 16 pro excluido
iphone 6s excluido
funda personalizable iphone 11-16 excluido
iphone 15 plus nuevo / bateria 100% excluido
iphone 16pro excluido
iphone 14 128gb nuevos excluido
funda magnetica iphone 16 promax/16 pro/16 plus/16 excluido
funda iphone 16 pro max excluido
ip

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df

,id,nombre,precio,estado,url
0,1068806810,iphone 16 128 verde precintado,"750,00",nuevo,https://es.wallapop.com/item/iphone-16-128-verde-precintado-1068806810
1,1068790457,iphone 16 256gb,"900,00",nuevo,https://es.wallapop.com/item/iphone-16-256gb-1068790457
2,1068768810,iphone 16 128 gb,"859,00",nuevo,https://es.wallapop.com/item/iphone-16-128-gb-1068768810
3,1068813492,iphone 16 128gb precintado,"770,00",nuevo,https://es.wallapop.com/item/iphone-16-128gb-precintado-1068813492
4,1067492193,iphone 16 pro max 256 gb,"1280,00",nuevo,https://es.wallapop.com/item/iphone-16-pro-max-256-gb-1067492193
...,...,...,...,...,...
706,1051832157,apple iphone 16 pro max 512 gb titanio negro,"1575,00",nuevo,https://es.wallapop.com/item/apple-iphone-16-pro-256-gb-titanio-desierto-1051832157
707,1067140874,iphone 16 pro max 256gb titanio precintado,"1449,00",nuevo,https://es.wallapop.com/item/iphone-16-pro-max-256gb-titanio-precintado-1067140874
708,1067140836,iphone 16 pro max 256gb blanco precintado,"1449,00",nuevo,https://es.wallapop.com/item/iphone-16-pro-max-256gb-blanco-precintado-1067140836
709,1067493221,iphone 16 nuevo,"839,99",nuevo,https://es.wallapop.com/item/iphone-16-nuevo-1067493221


In [ ]:
"""try:
    df['precio'] = df['precio'].str.replace(',', '.')
except:
    pass
df['precio'] = df['precio'].astype(float)
"""

"try:\n    df['precio'] = df['precio'].str.replace(',', '.')\nexcept:\n    pass\ndf['precio'] = df['precio'].astype(float)\n"

In [ ]:
hoy_formateada = datetime.now().strftime("%Y%m%d")
print(hoy_formateada)
print(elemento_a_buscar)
print(estado)

20241112
iphone 16
nuevo


In [ ]:
nombre_archivo_pkl = hoy_formateada + '_' + elemento_a_buscar + '_' + estado
df.to_pickle('pickels/' + nombre_archivo_pkl + '.pkl')

In [ ]:
try:
    df = pd.read_pickle('pickels/' + nombre_archivo_pkl + '.pkl')
    print("pickel guardado")
except:
    print("Error leyendo archivo! puede que no se haya guardado bien")
driver.quit()

pickel guardado
